In [ ]:
from programs.sigma_clipping import FastSigmaClipping
from programs.standard_deviation import FastStandardDeviation, GenericFilter
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt

filepath = '/home/avoyeux/Documents/work_codes/sigma_clipping_tests/results/solo_L1_spice-n-ras_20250801T121921_V01_335544522-000.fits'

In [ ]:
def show_data_image(data, lambda_index, title, min_max=None):
    data_ = np.copy(data)
    image = data_[lambda_index, :, :]
    plt.figure(figsize=(8, 6))
    if min_max is None:
        plt.imshow(image, origin='lower', cmap='inferno', aspect=0.25)  # aspect supposed to be 1/step
    else:
        plt.imshow(image, origin='lower', cmap='inferno', aspect=0.25, vmin=min_max[0], vmax=min_max[1])
    plt.colorbar(label='Intensity')
    plt.title(f"{title}")  #  (lambda_index={lambda_index})
    plt.show()


In [ ]:
hdul = fits.open(filepath)
data = hdul[1].data.astype(np.float64)
original_data = data[0]  # remove t dimension
print(f"data dtype is {data.dtype}")
lambda_index = int(original_data.shape[0] / 2)
print('Original data shape:', data.shape)
show_data_image(original_data, lambda_index, "Original data", min_max=(0, 3000))

In [ ]:
std_new = FastStandardDeviation(
    data=original_data,
    kernel=(3, 3, 3),
    with_NaNs=True,
).sdev
print(f"std dtype is {std_new.dtype}")
show_data_image(std_new, lambda_index, "Standard Deviation", min_max=(0, 3000))


In [ ]:
std_old = GenericFilter(
    data=original_data,
    kernel_size=3,
    with_nans=True,
).sdev
print(f"std dtype is {std_old.dtype}")
show_data_image(std_old, lambda_index, "Standard Deviation (GenericFilter)", min_max=(0, 3000))

In [ ]:
# DIFFERENCE

diff = std_new - std_old
show_data_image(diff, lambda_index, "Difference FastStandardDeviation - GenericFilter", min_max=(-0.1, 0.1))
